In [3]:
# ps47
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=c1ccb5d1025f668bfa7b6f49748d027d433266f7ca604977cd31e2ec23fbf40e
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


🌟 𝐏𝐘𝐒𝐏𝐀𝐑𝐊 𝐏𝐫𝐚𝐜𝐭𝐢𝐜𝐞/𝐈𝐧𝐭𝐞𝐫𝐯𝐢𝐞𝐰 𝐏𝐫𝐨𝐛𝐥𝐞𝐦 📊
========================================

Write an Pyspark code that reports the buyers who have bought S8 but not iPhone. Note that S8 and iPhone are products present in the Product table.

𝐒𝐜𝐡𝐞𝐦𝐚 𝐀𝐧𝐝 𝐃𝐚𝐭𝐚:
================

 # Define the schema for Product table
product_schema = StructType([
 StructField("product_id", IntegerType(), True),
 StructField("product_name", StringType(), True),
 StructField("unit_price", IntegerType(), True)
])
# Data for Product table
product_data = [(1, "S8", 1000),
 (2, "G4", 800),
 (3, "iPhone", 1400)]

# Define the schema for Sales table
sales_schema = StructType([
 StructField("seller_id", IntegerType(), True),
 StructField("product_id", IntegerType(), True),
 StructField("buyer_id", IntegerType(), True),
 StructField("sale_date", StringType(), True),
 StructField("quantity", IntegerType(), True),
 StructField("price", IntegerType(), True)
])
# Data for Sales table
sales_data = [(1, 1, 1, "2019-01-21", 2, 2000),
 (1, 2, 2, "2019-02-17", 1, 800),
 (2, 2, 3, "2019-06-02", 1, 800),
 (3, 3, 4, "2019-05-13", 2, 2800)]

In [4]:
# import necessary module
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
# create spark session
spark = SparkSession.builder.master('local').appName('pyspark-day-9').getOrCreate()
spark

In [6]:
 # Define the schema for Product table
product_schema = StructType([
 StructField("product_id", IntegerType(), True),
 StructField("product_name", StringType(), True),
 StructField("unit_price", IntegerType(), True)
])
# Data for Product table
product_data = [(1, "S8", 1000),
 (2, "G4", 800),
 (3, "iPhone", 1400)]

In [7]:
# Define the schema for Sales table
sales_schema = StructType([
 StructField("seller_id", IntegerType(), True),
 StructField("product_id", IntegerType(), True),
 StructField("buyer_id", IntegerType(), True),
 StructField("sale_date", StringType(), True),
 StructField("quantity", IntegerType(), True),
 StructField("price", IntegerType(), True)
])
# Data for Sales table
sales_data = [(1, 1, 1, "2019-01-21", 2, 2000),
 (1, 2, 2, "2019-02-17", 1, 800),
 (2, 2, 3, "2019-06-02", 1, 800),
 (3, 3, 4, "2019-05-13", 2, 2800)]

In [8]:
# create df for product
product_df = spark.createDataFrame(schema = product_schema, data = product_data)
product_df.show()

+----------+------------+----------+
|product_id|product_name|unit_price|
+----------+------------+----------+
|         1|          S8|      1000|
|         2|          G4|       800|
|         3|      iPhone|      1400|
+----------+------------+----------+



In [9]:
# create df for sales
sales_df = spark.createDataFrame(schema = sales_schema, data = sales_data)
sales_df.show()

+---------+----------+--------+----------+--------+-----+
|seller_id|product_id|buyer_id| sale_date|quantity|price|
+---------+----------+--------+----------+--------+-----+
|        1|         1|       1|2019-01-21|       2| 2000|
|        1|         2|       2|2019-02-17|       1|  800|
|        2|         2|       3|2019-06-02|       1|  800|
|        3|         3|       4|2019-05-13|       2| 2800|
+---------+----------+--------+----------+--------+-----+



In [10]:
product_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- unit_price: integer (nullable = true)



In [11]:
sales_df.printSchema()

root
 |-- seller_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- buyer_id: integer (nullable = true)
 |-- sale_date: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: integer (nullable = true)



In [12]:
# Join both df
result_df = product_df.join(sales_df, on='product_id', how='inner')
result_df.show()

+----------+------------+----------+---------+--------+----------+--------+-----+
|product_id|product_name|unit_price|seller_id|buyer_id| sale_date|quantity|price|
+----------+------------+----------+---------+--------+----------+--------+-----+
|         1|          S8|      1000|        1|       1|2019-01-21|       2| 2000|
|         2|          G4|       800|        1|       2|2019-02-17|       1|  800|
|         2|          G4|       800|        2|       3|2019-06-02|       1|  800|
|         3|      iPhone|      1400|        3|       4|2019-05-13|       2| 2800|
+----------+------------+----------+---------+--------+----------+--------+-----+



In [13]:
# filter out data for s8
s8_df = result_df.filter(col('product_name') == 'S8').select('buyer_id')
s8_df.show()

+--------+
|buyer_id|
+--------+
|       1|
+--------+



In [14]:
# filter out data for iphone
iphone_df = result_df.filter(col('product_name') == 'iPhone').select('buyer_id')
iphone_df.show()

+--------+
|buyer_id|
+--------+
|       4|
+--------+



In [15]:
# join df using left_anti
s8_buyer = s8_df.subtract(iphone_df).select('buyer_id')
s8_buyer.show()

+--------+
|buyer_id|
+--------+
|       1|
+--------+

